# Stablecoin Policy Research - Data Exploration

This notebook provides an initial exploration of the stablecoin market data and policy events.

## Objectives

1. **Data Overview**: Examine the structure and quality of collected data
2. **Market Behavior**: Analyze stablecoin price dynamics and peg deviations
3. **Policy Events**: Identify and categorize Federal Reserve policy events
4. **Preliminary Analysis**: Initial insights into market-policy relationships

## Data Sources

- **Stablecoin Prices**: CoinGecko API
- **Policy Events**: Federal Reserve announcements and speeches
- **Macro Indicators**: FRED database
- **Market Data**: Yahoo Finance


In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Import project modules
import sys
sys.path.append('../src')
from utils.config import load_config
from utils.io import load_data, save_data

# Load configuration
config = load_config('../configs/config.yaml')


## 1. Data Loading and Overview


In [ ]:
# Load stablecoin price data
try:
    stablecoin_prices = load_data('stablecoin_prices', config['data']['processed_dir'])
    print("Stablecoin prices loaded successfully")
    print(f"Shape: {stablecoin_prices.shape}")
    print(f"Date range: {stablecoin_prices.index.min()} to {stablecoin_prices.index.max()}")
    print(f"Stablecoins: {list(stablecoin_prices.columns)}")
except FileNotFoundError:
    print("Stablecoin price data not found. Run data ingestion first.")
    stablecoin_prices = None

# Load policy events data
try:
    policy_events = load_data('policy_events', config['data']['processed_dir'])
    print("\nPolicy events loaded successfully")
    print(f"Shape: {policy_events.shape}")
    print(f"Event types: {policy_events['event_type'].unique()}")
except FileNotFoundError:
    print("Policy events data not found. Run data ingestion first.")
    policy_events = None


## 2. Stablecoin Price Analysis


In [ ]:
if stablecoin_prices is not None:
    # Calculate peg deviations
    peg_deviations = stablecoin_prices - 1.0
    
    # Summary statistics
    print("Peg Deviation Statistics:")
    print(peg_deviations.describe())
    
    # Plot price evolution
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Price levels
    stablecoin_prices.plot(ax=axes[0, 0], title='Stablecoin Prices')
    axes[0, 0].axhline(y=1.0, color='red', linestyle='--', alpha=0.7)
    axes[0, 0].set_ylabel('Price (USD)')
    
    # Peg deviations
    peg_deviations.plot(ax=axes[0, 1], title='Peg Deviations')
    axes[0, 1].axhline(y=0, color='red', linestyle='--', alpha=0.7)
    axes[0, 1].set_ylabel('Deviation from $1.00')
    
    # Volatility (rolling standard deviation)
    volatility = stablecoin_prices.pct_change().rolling(30).std()
    volatility.plot(ax=axes[1, 0], title='30-Day Rolling Volatility')
    axes[1, 0].set_ylabel('Volatility')
    
    # Distribution of peg deviations
    peg_deviations.hist(bins=50, ax=axes[1, 1], alpha=0.7)
    axes[1, 1].set_title('Distribution of Peg Deviations')
    axes[1, 1].set_xlabel('Deviation from $1.00')
    
    plt.tight_layout()
    plt.show()
else:
    print("No stablecoin price data available for analysis")
